In [1]:
import tensorflow as tf
import numpy as np
import os
from sklearn.metrics import classification_report

In [2]:
dataset_path = f'{os.getcwd()}/drive/MyDrive/dataset/dataset_bisindo'
train_path = f'{dataset_path}/datatrain'
test_path = f'{dataset_path}/datatest'

In [3]:
try:
  device_name = tf.test.gpu_device_name().split(':')[1]
except:
  device_name = 'CPU'
finally:
  print(f'Using {device_name} for Tensorflow {tf.__version__}')

Using GPU for Tensorflow 2.4.1


In [4]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
train_generator = datagen.flow_from_directory(
    train_path,
    target_size=(512, 512),
    batch_size=32,
    class_mode='sparse'
)

test_generator = datagen.flow_from_directory(
    test_path,
    target_size=(512, 512),
    batch_size=32,
    class_mode='sparse',
    shuffle=False
)

Found 1727 images belonging to 27 classes.
Found 432 images belonging to 27 classes.


In [5]:
class AccuracyThreshold(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        self.threshold = threshold
          
    def on_epoch_end(self, epoch, logs={}):
        score = logs.get('accuracy')
        if score > self.threshold:
            print(f'\nReached {score:.1%} accuracy, stopping training!')
            self.model.stop_training = True

In [6]:
def create_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(512, 512, 3)))
    model.add(tf.keras.layers.MaxPooling2D(2, 2))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(2, 2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(27, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [7]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 510, 510, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 255, 255, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 253, 253, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 126, 64)      0         
_________________________________________________________________
flatten (Flatten)            (None, 1016064)           0         
_________________________________________________________________
dense (Dense)                (None, 256)               260112640 
_________________________________________________________________
dense_1 (Dense)              (None, 27)                6

In [8]:
history = model.fit(
    train_generator, 
    epochs=5,
    verbose=1,
    callbacks=[AccuracyThreshold(0.9901)]
)

Epoch 1/5
54/54 [==============================] - 108s 2s/step - loss: 11.8791 - accuracy: 0.2898
Epoch 2/5
54/54 [==============================] - 100s 2s/step - loss: 0.3711 - accuracy: 0.9280
Epoch 3/5
54/54 [==============================] - 100s 2s/step - loss: 0.0551 - accuracy: 0.9860
Epoch 4/5
54/54 [==============================] - 100s 2s/step - loss: 0.0330 - accuracy: 0.9920

Reached 99.2% accuracy, stopping training!


In [9]:
y_pred = np.argmax(model.predict(test_generator), axis=-1)
y_true = test_generator.classes
print(classification_report(y_true, y_pred, zero_division=0))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        16
           2       0.55      1.00      0.71        16
           3       0.67      0.62      0.65        16
           4       1.00      1.00      1.00        16
           5       1.00      1.00      1.00        16
           6       0.80      0.75      0.77        16
           7       0.89      1.00      0.94        16
           8       0.74      0.88      0.80        16
           9       0.20      0.06      0.10        16
          10       0.64      1.00      0.78        16
          11       0.67      0.25      0.36        16
          12       0.44      0.44      0.44        16
          13       0.42      0.31      0.36        16
          14       0.80      1.00      0.89        16
          15       0.94      1.00      0.97        16
          16       0.82      0.56      0.67        16
          17       0.80    